<a href="https://colab.research.google.com/github/vincentjunitio00/Rock-Paper-Scizzor-Image-Recognition/blob/master/Rock_Paper_Scizzor_Self_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>